In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

## IO markup

### Train

In [2]:
from modules.data import bert_data

In [3]:
data = bert_data.LearnData.create(
    train_df_path="/home/ubuntu/factRuEval-2016/dev.csv",
    valid_df_path="/home/ubuntu/factRuEval-2016/test.csv",
    idx2labels_path="/home/ubuntu/factRuEval-2016/idx2labels.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [4]:
from modules.models.bert_models import BERTBiLSTMCRF

In [5]:
model = BERTBiLSTMCRF.create(len(data.train_ds.idx2label), lstm_dropout=0., crf_dropout=0.3, hidden_dim=256)

In [6]:
from modules.train.train import NerLearner

In [9]:
num_epochs = 100

In [10]:
learner = NerLearner(
    model, data, "/home/ubuntu/models/fre-BERTBiLSTMCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [11]:
model.get_n_trainable_params()

953593

In [14]:
learner.fit(epochs=num_epochs)

INFO:root:Resuming train... Current epoch 72.


INFO:root:
epoch 73, average train epoch loss=4.9766e-05



INFO:root:on epoch 67 by max_f1: 0.904


              precision    recall  f1-score   support

         I_O      0.979     0.987     0.983     48875
       I_LOC      0.905     0.818     0.859      1557
       I_ORG      0.857     0.766     0.809      3865
       I_PER      0.957     0.962     0.959      2112

   micro avg      0.969     0.967     0.968     56409
   macro avg      0.925     0.883     0.903     56409
weighted avg      0.968     0.967     0.967     56409



### Eval

#### Load model

In [15]:
from modules.data import bert_data

In [16]:
data = bert_data.LearnData.create(
    train_df_path="/home/ubuntu/factRuEval-2016/dev.csv",
    valid_df_path="/home/ubuntu/factRuEval-2016/test.csv",
    idx2labels_path="/home/ubuntu/factRuEval-2016/idx2labels.txt",
    clear_cache=False
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [17]:
from modules.models.bert_models import BERTBiLSTMCRF

In [18]:
model = BERTBiLSTMCRF.create(len(data.train_ds.idx2label), lstm_dropout=0., crf_dropout=0.3, hidden_dim=256)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/ubuntu/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/ubuntu/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpea90pt0v
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pool

In [19]:
from modules.train.train import NerLearner

In [20]:
learner = NerLearner(model, data, "/home/ubuntu/models/fre-BERTBiLSTMCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [21]:
model.get_n_trainable_params()

953593

In [22]:
learner.load_model()

### Predict

In [23]:
from modules.data.bert_data import get_data_loader_for_predict

In [24]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [25]:
preds = learner.predict(dl)

In [26]:
from sklearn_crfsuite.metrics import flat_classification_report

In [27]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [42]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [43]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC"], digits=4)

In [44]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8490    0.7840    0.8152      3865
       I_PER     0.9562    0.9602    0.9582      2112
       I_LOC     0.8992    0.8253    0.8607      1557

   micro avg     0.8910    0.8419    0.8658      7534
   macro avg     0.9015    0.8565    0.8780      7534
weighted avg     0.8894    0.8419    0.8647      7534



In [62]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [63]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.8002    0.7391    0.7684      1311
           O     0.9755    0.9852    0.9803     48686
         ORG     0.7244    0.5951    0.6534      1877
         PER     0.8570    0.8254    0.8409      1300

    accuracy                         0.9614     53174
   macro avg     0.8393    0.7862    0.8108     53174
weighted avg     0.9594    0.9614    0.9601     53174



## BIO markup

### Train

In [64]:
from modules.data import bert_data

In [65]:
data = bert_data.LearnData.create(
    train_df_path="/home/ubuntu/factRuEval-2016/dev.csv",
    valid_df_path="/home/ubuntu/factRuEval-2016/test.csv",
    idx2labels_path="/home/ubuntu/factRuEval-2016/idx2labels_BIO.txt",
    clear_cache=True,
    markup="BIO"
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [66]:
from modules.models.bert_models import BERTBiLSTMCRF, BERTBiLSTMNCRF

In [68]:
model = BERTBiLSTMCRF.create(len(data.train_ds.idx2label), lstm_dropout=0., crf_dropout=0.3, hidden_dim=512)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/ubuntu/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/ubuntu/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpktkqtzsx
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pool

In [69]:
from modules.train.train import NerLearner

In [72]:
num_epochs = 100

In [73]:
learner = NerLearner(model, data, "/home/ubuntu/models/fre-BERTBiLSTMCRF-BIO.cpt", t_total=num_epochs * len(data.train_dl))

In [74]:
model.get_n_trainable_params()

2235585

In [76]:
learner.fit(epochs=num_epochs)

INFO:root:Resuming train... Current epoch 44.


INFO:root:
epoch 45, average train epoch loss=0.008046



INFO:root:on epoch 33 by max_f1: 0.849


              precision    recall  f1-score   support

         B_O      0.982     0.985     0.984     48875
       B_LOC      0.893     0.820     0.855      1320
       B_ORG      0.835     0.757     0.794      1910
       I_ORG      0.766     0.785     0.775      1955
       B_PER      0.923     0.941     0.932      1319
       I_PER      0.959     0.951     0.955       793
       I_LOC      0.805     0.540     0.646       237

   micro avg      0.966     0.963     0.965     56409
   macro avg      0.881     0.825     0.849     56409
weighted avg      0.965     0.963     0.964     56409



### Eval

#### Load model

In [79]:
from modules.data import bert_data

In [103]:
data = bert_data.LearnData.create(
    train_df_path="/home/ubuntu/factRuEval-2016/dev.csv",
    valid_df_path="/home/ubuntu/factRuEval-2016/test.csv",
    idx2labels_path="/home/ubuntu/factRuEval-2016/idx2labels_BIO.txt",
    clear_cache=False,
    markup="BIO"
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/ubuntu/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [104]:
from modules.models.bert_models import BERTBiLSTMCRF

In [105]:
model = BERTBiLSTMCRF.create(len(data.train_ds.idx2label), lstm_dropout=0., crf_dropout=0.3, hidden_dim=512)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/ubuntu/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/ubuntu/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmppbrjc9tb
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pool

In [106]:
from modules.train.train import NerLearner

In [107]:
learner = NerLearner(model, data, "/home/ubuntu/models/fre-BERTBiLSTMCRF-BIO.cpt", t_total=num_epochs * len(data.train_dl))

In [108]:
model.get_n_trainable_params()

2235585

In [109]:
learner.load_model()

### Predict

In [110]:
from modules.data.bert_data import get_data_loader_for_predict

In [111]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [113]:
preds = learner.predict(dl)

In [114]:
from sklearn_crfsuite.metrics import flat_classification_report

In [115]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [116]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [123]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels,
                                           labels=["I_ORG", "I_PER", "I_LOC", "B_ORG", "B_PER", "B_LOC"], digits=4)

In [124]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.8183    0.7488    0.7821      1955
       I_PER     0.9619    0.9546    0.9582       793
       I_LOC     0.8065    0.5274    0.6378       237
       B_ORG     0.8297    0.7424    0.7836      1910
       B_PER     0.9357    0.9371    0.9364      1319
       B_LOC     0.8814    0.8386    0.8595      1320

   micro avg     0.8703    0.8106    0.8394      7534
   macro avg     0.8722    0.7915    0.8263      7534
weighted avg     0.8675    0.8106    0.8370      7534



In [119]:
from modules.analyze_utils.plot_metrics import get_f1_score

In [120]:
get_f1_score(true_labels, pred_labels, labels=["I_ORG", "I_PER", "I_LOC", "B_ORG", "B_PER", "B_LOC"])

0.8262519680452184

In [121]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [122]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.7840    0.7165    0.7487      1312
           O     0.9749    0.9851    0.9800     48711
         ORG     0.7326    0.5948    0.6565      1888
         PER     0.8504    0.8204    0.8352      1303

    accuracy                         0.9606     53214
   macro avg     0.8355    0.7792    0.8051     53214
weighted avg     0.9585    0.9606    0.9592     53214

